In [104]:

!jt -t monokai

# Important Imports

In [13]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
from qiskit.circuit import QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.circuit.library import QFT
from math import gcd, ceil, log2
import numpy as np
from fractions import Fraction
from IPython.display import display
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator
import requests
import random  
# Import the required modules
from qiskit.circuit import QuantumCircuit
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit.visualization import plot_histogram
from matplotlib import pyplot as plt

import QuantumRingsLib
from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2
from quantumrings.toolkit.qiskit import QrJobV1

from matplotlib import pyplot as plt

# Acquire the Quantum Rings Provider
# qr_provider = QuantumRingsProvider(token =<YOUR_TOKEN_HERE>, name=<YOUR_ACCOUNT_NAME_HERE>)
qr_provider = QuantumRingsProvider(
    token='rings-128.zLEHHuVHp0o1quzabhgWkUZjZwXGxCHj',
    name='mn.mansouri@esi-sba.dz'
)
# backend = provider.get_backend("scarlet_quantum_rings")

# Getting Started

#### 1. Number to Factorize (`N`)

The number `N` is the integer that we want to factorize. This value is chosen as an input to the quantum algorithm. For this example, `N` is set to `15`, but in practice, it can be any positive integer that needs factorization.

#### 2. Number of Bits (`n`)

The number of bits `n` represents how many bits are needed to represent the number `N` in binary form. This is computed by taking the ceiling of the logarithm (base 2) of `N`, which ensures that the number of bits is sufficient to represent the value of `N`. For example, `15` requires 4 bits (`n = 4`), as `np.ceil(np.log2(15))` results in `4`.

#### 3. Quantum Registers

Quantum registers are used to store quantum bits (qubits) during quantum operations. The setup involves two main quantum registers:

- **Input Quantum Register (`q`)**: This register holds the qubits that represent the input quantum state. The size of this register is `2n`, which is chosen to accommodate both the quantum state and the operations that will be performed during the algorithm. The factor of 2 comes from the typical structure of many quantum algorithms, where two registers are used for the input and output states.

- **Target Quantum Register (`t`)**: This register contains `n` qubits, which are used for the target state in quantum operations. It stores intermediate and final quantum states during the computation.

#### 4. Classical Register (`c`)

After quantum operations are performed, the final results need to be measured and stored. A classical register is used for this purpose. The classical register holds `2n` classical bits that store the measurement results of the quantum states. These results are obtained after performing quantum measurements on the quantum registers, and they are essential for interpreting the outcome of the quantum computation.

In [14]:
N = 15 # number to factorize
n = np.ceil(np.log2(N)).astype(int) # number of bits to represent N
q = QuantumRegister(2*n, "q") # input qubits
t = QuantumRegister(n, "t") # target qubits
c = ClassicalRegister(2*n, "c") # classical bits
print(n)
print(t)
print(c)

4
QuantumRegister(4, 't')
ClassicalRegister(8, 'c')


### Selecting a Base `a` for Quantum Factorization

This process selects a random integer `a` that is coprime with a number `N` to be used in a quantum factoring algorithm. The base `a` must satisfy the condition that `gcd(a, N) = 1`.

#### Steps:

1. **Randomly select `a`**: Choose `a` from the range `[2, N)`.
2. **Check coprimeness**: If `gcd(a, N) > 1`, choose a new `a`.
3. **Repeat** until a valid `a` is found.
4. **Output** the chosen base `a`.

This ensures that `a` and `N` are coprime, which is essential for algorithms like Shor's algorithm to work correctly.


In [15]:
a = np.random.randint(2, N) # number possibly coprime with N
while gcd(a, N) > 1:
    a = np.random.randint(2, N)
print(f"Chosen base a = {a}")
a = 7

Chosen base a = 13


### Building Circuit

#### `Uf` Function for Quantum Factorization

The `Uf` function defines a quantum operation for modular exponentiation, which is used in algorithms like Shor's algorithm. This function applies a series of controlled-phase gates based on the powers of `a` modulo `N`.

##### Function Overview

- **Purpose**: The function applies modular exponentiation to the quantum register, which is a core operation for quantum factoring algorithms.
- **Input**: A quantum circuit `circ` which has the necessary quantum registers (`q` and `t`) set up.
- **Operation**: It iterates over the range `2*n` (where `n` is the number of bits representing `N`) and applies a controlled-phase gate to the quantum register for each power of `a`, computed modulo `N`.

##### Steps in the Function

1. **Modular Exponentiation**: For each iteration, it computes `a^(2^i) % N` using `pow(a, 2**i, N)`.
2. **Controlled-Phase Gates**: The function applies a controlled-phase gate to the quantum register. The phase is determined by `2 * np.pi * mod_exp / N`, where `mod_exp` is the result of the modular exponentiation.
3. **Iteration**: This process repeats for `i` in the range `[0, 2*n)`.

##### Key Concepts

- **Controlled-Phase Gate** (`cp`): A gate that applies a phase shift based on the control qubit.
- **Modular Exponentiation**: A fundamental operation in Shor's algorithm that is used to find periodicity in quantum systems.


In [16]:
def Uf(circ):
    for i in range(2*n):
        mod_exp = pow(a, 2**i, N)
        circ.cp(2*np.pi*mod_exp/N, q[i], t)

### Quantum Phase Estimation (QPE) Circuit

This document outlines the steps involved in constructing a **Quantum Phase Estimation (QPE)** circuit. The QPE algorithm is used for estimating the eigenvalue (or phase) of a unitary operator. It is an essential component in many quantum algorithms, such as **Shor's algorithm** for factoring large numbers.

#### QPE Circuit Breakdown

##### 1. **Initialization**
The circuit begins by initializing the quantum registers:
- `q`: The input qubits (quantum register).
- `t`: The target qubits (quantum register).
- `c`: The classical register.

The initialization steps are as follows:
- `qpe.x(t)`: The target qubits are initialized to the state \( \lvert 1 \rangle \).
- `qpe.h(q)`: A Hadamard gate is applied to all qubits in the input register `q` to create an equal superposition of all possible states.

##### 2. **Modular Exponentiation**
The core of the QPE circuit involves applying the modular exponentiation, which encodes the phase information in the target register `t`. This step is performed by the `Uf` function, which applies modular exponentiation based on the chosen base `a` and the number `N` to be factored.

##### 3. **Quantum Fourier Transform (QFT)**
After applying modular exponentiation, the quantum state in the input register needs to be transformed to extract the phase. This is achieved by applying the **Quantum Fourier Transform (QFT)**:
- `qpe.append(QFT(2*n, inverse=True), q)`: The inverse QFT is applied to the input qubits in the register `q` to convert the phase information into a measurable format.

##### 4. **Measurement**
Finally, the quantum state is measured:
- `qpe.measure(range(n*2), range(n*2))`: This command measures all qubits and stores the results in the classical register `c`. The measurement collapses the quantum state, extracting the eigenvalue (phase) from the quantum state.

##### 5. **Drawing the Circuit**
- `qpe.draw()`: This command visually represents the quantum circuit, showing all qubits, gates, and measurements in the QPE process.

#### Purpose of QPE in Quantum Algorithms

In algorithms like Shor’s, QPE is used to estimate the phase of a modular exponentiation operator, which is essential for determining the periodicity of a function. The phase information is then used to identify non-trivial factors of a number, enabling quantum algorithms to solve problems that are difficult for classical computers.

#### Summary

The QPE circuit applies several key quantum operations:
- Hadamard gates to create superposition,
- Modular exponentiation to encode phase information,
- Quantum Fourier Transform to extract the phase,
- Measurement to obtain classical output.

This setup is central to algorithms that require phase estimation, such as Shor's algorithm for factoring large numbers.


In [17]:
qpe = QuantumCircuit(q, t, c, name="quantum phase estimation")
qpe.x(t) # initialize the target qubits to |1>
qpe.h(q) # apply H-gate to all input qubits
Uf(qpe) # apply modular exponentiation
qpe.append(QFT(2*n, inverse=True), q) # apply the inverse QFT
qpe.measure(range(n*2), range(n*2))
qpe.draw()

┌───┐                                                                     »
q_0: ┤ H ├─■───────────■──────────────────────■────────────────────────────────»
     ├───┤ │           │                      │                                »
q_1: ┤ H ├─┼───────────┼───────────■──────────┼───────────■────────────────────»
     ├───┤ │           │           │          │           │                    »
q_2: ┤ H ├─┼───────────┼───────────┼──────────┼───────────┼──────────■─────────»
     ├───┤ │           │           │          │           │          │         »
q_3: ┤ H ├─┼───────────┼───────────┼──────────┼───────────┼──────────┼─────────»
     ├───┤ │           │           │          │           │          │         »
q_4: ┤ H ├─┼───────────┼───────────┼──────────┼───────────┼──────────┼─────────»
     ├───┤ │           │           │          │           │          │         »
q_5: ┤ H ├─┼───────────┼───────────┼──────────┼───────────┼──────────┼─────────»
     ├───┤ │           │           │          │           │          │         »
q_6: ┤ H ├─┼───────────┼───────────┼──────────┼───────────┼──────────┼─────────»
     ├───┤ │           │           │          │           │          │         »
q_7: ┤ H ├─┼───────────┼───────────┼──────────┼───────────┼──────────┼─────────»
     ├───┤ │P(14π/15)  │           │P(8π/15)  │           │          │P(2π/15) »
t_0: ┤ X ├─■───────────┼───────────■──────────┼───────────┼──────────■─────────»
     ├───┤             │P(14π/15)             │           │P(8π/15)            »
t_1: ┤ X ├─────────────■──────────────────────┼───────────■────────────────────»
     ├───┤                                    │P(14π/15)                       »
t_2: ┤ X ├────────────────────────────────────■────────────────────────────────»
     ├───┤                                                                     »
t_3: ┤ X ├─────────────────────────────────────────────────────────────────────»
     └───┘                                                                     »
c: 8/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«                                                                        »
«q_0: ─■─────────────────────────────────────────────────────────────────»
«      │                                                                 »
«q_1: ─┼───────────■────────────────────────────────■────────────────────»
«      │           │                                │                    »
«q_2: ─┼───────────┼──────────■─────────────────────┼──────────■─────────»
«      │           │          │                     │          │         »
«q_3: ─┼───────────┼──────────┼──────────■──────────┼──────────┼─────────»
«      │           │          │          │          │          │         »
«q_4: ─┼───────────┼──────────┼──────────┼──────────┼──────────┼─────────»
«      │           │          │          │          │          │         »
«q_5: ─┼───────────┼──────────┼──────────┼──────────┼──────────┼─────────»
«      │           │          │          │          │          │         »
«q_6: ─┼───────────┼──────────┼──────────┼──────────┼──────────┼─────────»
«      │           │          │          │          │          │         »
«q_7: ─┼───────────┼──────────┼──────────┼──────────┼──────────┼─────────»
«      │           │          │          │P(2π/15)  │          │         »
«t_0: ─┼───────────┼──────────┼──────────■──────────┼──────────┼─────────»
«      │           │          │P(2π/15)             │          │         »
«t_1: ─┼───────────┼──────────■─────────────────────┼──────────┼─────────»
«      │           │P(8π/15)                        │          │P(2π/15) »
«t_2: ─┼───────────■────────────────────────────────┼──────────■─────────»
«      │P(14π/15)                                   │P(8π/15)            »
«t_3: ─■────────────────────────────────────────────■────────────────────»
«                 

### Retrievin Denominator

#### Function: `retrieve_denominator`

The `retrieve_denominator` function sends a POST request to a local web service to fetch the denominator for a given `decimal_value`, `n`, and `N`. 

##### Steps:
1. Convert inputs (`decimal_value`, `n`, `N`) to integers.
2. Send a POST request to `http://127.0.0.1:5000/denominator` with these values.
3. If the request is successful, return the denominator from the response.
4. If an error occurs, return an error message.


In [18]:
def retrieve_denominator(decimal_value, n, N):
    decimal_value = int(decimal_value)
    n = int(n)
    N = int(N)
    
    url = "http://127.0.0.1:5000/denominator"
    payload = {
        "decimal_value": decimal_value,
        "n": n,
        "N": N
    }
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            return response.json().get("denominator")
        else:
            return f"Error: {response.status_code}, {response.json().get('error', 'Unknown error')}"
    except Exception as e:
        return f"Request failed: {str(e)}"


# Testing Everything

In [19]:
N = 15 # number to factorize
N = int(N)
n = ceil(log2(N))  # number of bits to represent N
q = QuantumRegister(2*n, "q")
t = QuantumRegister(n, "t")
c = ClassicalRegister(2*n, "c")
state = False
service = QiskitRuntimeService()


In [20]:
def Uf(circ):
    for i in range(2*n):
        base = int(a)
        exp = int(2**i)
        mod = int(N)
        mod_exp = pow(base, exp, mod)
        circ.cp(2*np.pi*float(mod_exp)/float(N), q[i], t)

### Ensure denominator.py is running.

In [24]:
from quantumrings.toolkit.qiskit import QrBackendV2

while not state:
    a = random.randint(2, N-1)
    a = 7
    
    while gcd(a, N) > 1:
        a = random.randint(2, N-1)
        a = 7
    
    print(f"Chosen base a = {a}")
    
    
    qpe = QuantumCircuit(q, t, c)
    qpe.x(t)
    qpe.barrier()
    qpe.h(q)
    Uf(qpe)
    qpe.append(QFT(2*n, inverse=True), q)
    qpe.measure(range(n*2), range(n*2))
    qpe.barrier()
    qpe.measure(q, c)

    backend = QrBackendV2(qr_provider, num_qubits = qpe.num_qubits)

    
#     compiled = transpile(qpe, backend)
    compiled = transpile(qpe, backend, initial_layout=[i for i in range(0, qpe.num_qubits)])

    result = backend.run(compiled, shots=1024).result()
    counts = result.get_counts()
    sorted_measurements = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    
    for measurement, _ in sorted_measurements:
        decimal_value = int(measurement, 2)
        r = retrieve_denominator(decimal_value, n, N)
        
        if isinstance(r, int) and r % 2 == 0:
            x = pow(a, r // 2, N)
            factors = (gcd(x - 1, N), gcd(x + 1, N))
            
            if (factors[0] != 1 or factors[1] != 1) and (factors[0] != N or factors[1] != N):
                if factors[0] * factors[1] == N:
                    print(f"The factors for {N} are: {factors[0]}, {factors[1]}")
                    state = True
                    break
                elif factors[0] != 1 and ((N // factors[0]) * factors[0] == N):
                    print(f"The factors for {N} are: {factors[0]}, {N // factors[0]}")
                    state = True
                    break
                elif factors[1] != 1 and ((N // factors[1]) * factors[1] == N):
                    print(f"The factors for {N} are: {factors[1]}, {N // factors[1]}")
                    state = True
                    break

Chosen base a = 7


RuntimeError: Invalid Parameter passed to the gate instruction.